<a href="https://colab.research.google.com/github/virf96/Coppel/blob/main/pipeline_de_datos_para_entrenamiento_version_3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pipeline de datos
Este notebook contiene todas las funciones necesarias para crear un dataset de training y evaluacion de nuestro modelo de churn.

Esta dividido en tres partes, seteo de parametros, obtencion de datos, y contstruccion del dataset. 

## Obtencion de datos

In [ ]:
TABLA_VENTAS: str = "MUESTRA_TRANSACCIONES_VENTAS"
TABLA_HIST_CORTE: str = "MUESTRA_HISTORIAL_CORTE"
TABLA_DESERCION: str = "MUESTRA_DATADESERCION"
TABLA_SOCIODEMOGRAFICA: str = "MUESTRA_SOCIODEMOGRAFICA"
TABLA_DEVOLUCIONES: str = "DEVOLUCIONES"
FECHA_PRIMER_DIA: str = "2020-07-01"
VARIABLES_OBJETIVO=True
buffer=2
meses_pre=12
meses_post=12
submuestra = None
fechas_festivas = {
    "navidad": "2019-12-25",
    "dia_madres": "2019-05-10",
    "regreso_clases": "2019-08-20",
    "semana_santa": "2020-04-12"}

In [ ]:
import calendar
import datetime
import jaydebeapi
import numpy as np
import pandas as pd

from dateutil.relativedelta import relativedelta

from project_lib import Project

In [ ]:
pd.set_option("display.max_columns", None)


# Credenciales del proyecto
project = Project.access()
NZSQL_DIRECCIONRIESGOS_credentials = project.get_connection(
    name="NZSQL_DIRECCIONRIESGOS"
)

# Conexión a la base de datos
url_con = "{}://{}:{}/{}".format(
    "jdbc:netezza",
    NZSQL_DIRECCIONRIESGOS_credentials["host"],
    NZSQL_DIRECCIONRIESGOS_credentials["port"],
    NZSQL_DIRECCIONRIESGOS_credentials["database"],
)

# Establecer conexión a la base de datos desde pandas
NZSQL_DIRECCIONRIESGOS_connection = jaydebeapi.connect(
    'org.netezza.Driver',
    url_con,
    [NZSQL_DIRECCIONRIESGOS_credentials['username'],
    NZSQL_DIRECCIONRIESGOS_credentials['password']])

In [ ]:
def get_restriccion(
    tabla: str = "MUESTRA_TRANSACCIONES_VENTA",
    campo_fecha: str = "FECHACOMPRA",
    fecha_min: str = "",
    fecha_max: str = "",
    submuestra: int = None,
    campo_cliente: str = "IDCTE",
):
    """Obtener una muestra de la tabla.
    Se pueden poner restricciones vinculadas al campo fecha, o al campo cliente.

    """
    if tabla != "MUESTRA_SOCIODEMOGRAFICA":
        if submuestra is None:
            muestra = f"""({campo_fecha} >= '{fecha_min}' AND {campo_fecha} < '{fecha_max}')"""
        else:
            muestra = f"""({campo_fecha} >= '{fecha_min}'AND {campo_fecha} < '{fecha_max}'AND {campo_cliente} LIKE '%{submuestra}5')"""
    else:
        if submuestra is None:
            muestra = None
        else:
            muestra = f"""({campo_cliente} LIKE '%{submuestra}5') """
    return muestra

### Procesamiento transacciones

In [ ]:
def get_monto_compra_numero_productos_mes(
    restriccion: str,
    tabla: str = TABLA_VENTAS,
    NZSQL_DIRECCIONRIESGOS_connection=NZSQL_DIRECCIONRIESGOS_connection,
):
    consulta = f"""
    WITH venta_dia AS (
        SELECT CLIENTECODIGO,
            FECHACOMPRA,
            SUM(PRECIO_VTA) AS precio_venta_factura,
            COUNT(CODIGO) AS items_venta_factura
        FROM {tabla}
        WHERE {restriccion}
        GROUP BY CLIENTECODIGO, FECHACOMPRA
        )
    SELECT CLIENTECODIGO,
        DATE_PART('MONTH',FECHACOMPRA) AS MES,
        DATE_PART('YEAR',FECHACOMPRA) AS ANIO,
        COUNT(DISTINCT(FECHACOMPRA)) AS numero_compras,
        SUM(precio_venta_factura) AS monto_compra_total,
        AVG(precio_venta_factura) AS monto_compra_promedio,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY precio_venta_factura) AS monto_compra_mediana,
        MAX(precio_venta_factura) AS monto_compra_maximo,
        MIN(precio_venta_factura) AS monto_compra_minimo,
        SUM(items_venta_factura) AS items_compra_total,
        AVG(items_venta_factura) AS items_compra_promedio,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY items_venta_factura) AS items_compra_mediana,
        MAX(items_venta_factura) AS items_compra_maximo,
        MIN(items_venta_factura) AS items_compra_minimo
        FROM venta_dia
        GROUP BY CLIENTECODIGO, MES, ANIO
        ORDER BY ANIO, MES, CLIENTECODIGO
    """
    df = pd.read_sql(consulta, con=NZSQL_DIRECCIONRIESGOS_connection)
    df = df.rename(columns = str.lower).rename(columns = {'clientecodigo':'CLIENTECODIGO', 'mes':'MES', 'anio':'ANIO'})
    return df


def agregar_mes_relativo(
    df, fecha_de_corte, columna_anio="ANIO", columna_mes="MES"
):
    df["mes_relativo"] = np.round(
        (
            pd.to_datetime(
                [
                    "{}-{}-01".format(y, m)
                    for y, m in zip(df[columna_anio], df[columna_mes])
                ]
            )
            - pd.to_datetime(fecha_de_corte)
        ).days
        / 30,
        0,
    )
    return df


def generar_datos_meses_anteriores(df, indice="CLIENTECODIGO"):
    # Funcion para generar un dataset con una fila por cliente, con los datos
    # agregados de cada mes, con fecha de corte establecida en fecha_de_corte

    df = (
        df.drop(columns=["MES", "ANIO"])
        .pivot(index=indice, columns="mes_relativo")
        .reset_index()
    )
    df.columns = ["".join([col[0], str(col[1])]) for col in df.columns]
    return df


def generar_variables_objetivo(df, indice="CLIENTECODIGO"):
    # Funcion que devuelve un dataframe con las cuatro variables objetivo calculadas

    # Variables a calcular. Key es el nombre, value es una tupla con el rango de meses a calcular, no inclusive
    periodos = {
        "compra_1": (0, 2),
        "compra_3": (0, 4),
        "compra_6": (0, 7),
        "compra_12": (0, 13),
    }

    dfs = []
    for periodo in periodos:
        periodo_df = (
            df[
                (df["mes_relativo"] > periodos[periodo][0])
                & (df["mes_relativo"] < periodos[periodo][1])
            ][[indice, "numero_compras"]]
            .groupby(indice)["numero_compras"]
            .sum()
            > 0
        ).to_frame(name=periodo)
        dfs.append(periodo_df)

    return pd.concat(dfs, axis=1).reset_index().fillna(False)


def generar_estadisticas_compras(df, indice="CLIENTECODIGO"):
    variables = ["numero_compras", "monto_compra_total", "items_compra_total"]
    df_pasado = df[(df["mes_relativo"] < -1)]

    dfs = []
    for variable in variables:

        group = df_pasado[[indice, variable]].groupby(indice)[variable]
        dfs.append(group.mean().to_frame(f"promedio_{variable}"))
        dfs.append(group.median().to_frame(f"mediana_{variable}"))
        dfs.append(group.max().to_frame(f"max_{variable}"))

    return pd.concat(dfs, axis=1).reset_index()


def preparar_df_ventas(
    df,
    fecha_de_corte,
    columna_anio="ANIO",
    columna_mes="MES",
    indice="CLIENTECODIGO",
    variables_objetivo=False,
):
    # A partir de una tabla de ventas por mes y cliente, genera un dataframe con un cliente por row,
    # con sus datos agregados para cada mes, y la variable objetivo

    # Agrego el mes relativo a la fecha de corte
    df = agregar_mes_relativo(df, fecha_de_corte = fecha_de_corte, columna_anio = columna_anio, columna_mes = columna_mes )

    if variables_objetivo:
        # Creo un dataframe con las variables objetivo a 1, 3, 6 y 12 meses
        df_variables_objetivo = generar_variables_objetivo(df, indice)

    # Genero el promedio/max/mediana de la cantidad de items/cantidad de compras/monto total por mes
    estadisticas_compras_agregadas = generar_estadisticas_compras(df, indice)

    # Con las variables objetivo construidas, elimino todas las filas de despues de la fecha de corte
    # Elimino el mes inmediatamente anterior a la fecha de corte por el delay de datos en produccion
    df = df[df["mes_relativo"] < -1]

    # Agrego los datos de los meses anteriores
    df = generar_datos_meses_anteriores(df)

    if variables_objetivo:
        # Finalmente, mergeo los dataframes para crear el conjunto de datos completo
        df_final_ventas = pd.merge(df, df_variables_objetivo, on=indice, how="left")
        df_final_ventas[
            ["compra_1", "compra_3", "compra_6", "compra_12"]
        ] = df_final_ventas[["compra_1", "compra_3", "compra_6", "compra_12"]].fillna(
            value=False
        )
    else:
        df_final_ventas = df

    return df_final_ventas

## Fechas relevantes

In [ ]:
def ventas_dia(restriccion_historica_compra, tabla: str = TABLA_VENTAS):
    consulta = f"""
    SELECT CLIENTECODIGO,
        FECHACOMPRA,
        SUM(PRECIO_VTA) AS precio_venta_factura,
        COUNT(CODIGO) AS items_venta_factura
        FROM {tabla}
        WHERE {restriccion_historica_compra}
        GROUP BY CLIENTECODIGO, FECHACOMPRA
    """
    return pd.read_sql(consulta, con=NZSQL_DIRECCIONRIESGOS_connection)


def ponderar_variable_distancia_fecha(
    df, fecha, output_col_name, variable="items_venta_factura", k=0.8
):
    df = df.groupby(["CLIENTECODIGO", "FECHACOMPRA"])[variable].sum().reset_index()
    df["FECHACOMPRA"] = pd.to_datetime(df["FECHACOMPRA"])
    df["datediff"] = pd.to_datetime(fecha) - df["FECHACOMPRA"]
    df.loc[df.datediff < pd.to_timedelta(1, "d"), "datediff"] = pd.to_timedelta(-1, "d")
    df["weight"] = 1 / df.datediff.dt.days ** k
    df[output_col_name] = df.weight * df[variable]
    condition = df.datediff > pd.to_timedelta(0, "d")
    df[output_col_name].where(condition, 0, inplace=True)
    
    return df.set_index("CLIENTECODIGO").loc[:, output_col_name]


def get_variables_fechas_relevantes(fechas, restriccion_historica_compra, indice="CLIENTECODIGO", tabla=TABLA_VENTAS):
    df_dia = ventas_dia(tabla=TABLA_VENTAS, restriccion_historica_compra = restriccion_historica_compra)
    variable = "ITEMS_VENTA_FACTURA"
    df_dia_fechas = []
    for nombre_fecha, fecha in fechas.items():
        df_dia_fechas.append(
            ponderar_variable_distancia_fecha(df_dia, fecha, nombre_fecha, variable)
        )

    df_clientes = (
        pd.concat(df_dia_fechas, axis=1).groupby(indice)[list(fechas.keys())].sum()
    )
    return df_clientes.reset_index()

### Canal compras

In [ ]:
def get_cliente_canal_venta_prop(
    restriccion: str,
    tabla: str = TABLA_VENTAS,
    NZSQL_DIRECCIONRIESGOS_connection=NZSQL_DIRECCIONRIESGOS_connection,
):
    # Canales por cliente
    condicion_canal = f"""
    CASE WHEN SUM(DISTINCT(FLAG_FISICO)) = 1 AND SUM(DISTINCT(FLAG_DIGITAL)) = 0 THEN 'FISICO' 
         WHEN SUM(DISTINCT(FLAG_FISICO)) = 0 AND SUM(DISTINCT(FLAG_DIGITAL)) = 1 THEN 'DIGITAL'
         WHEN SUM(DISTINCT(FLAG_FISICO)) = 1 AND SUM(DISTINCT(FLAG_DIGITAL)) = 1 THEN 'AMBOS'
         ELSE 'NA' END
    """
    condicion_frecuencia_canal = f"""
    CASE WHEN SUM(DISTINCT(FLAG_FISICO)) = 1 AND SUM(DISTINCT(FLAG_DIGITAL)) = 0 THEN 1
         WHEN SUM(DISTINCT(FLAG_FISICO)) = 0 AND SUM(DISTINCT(FLAG_DIGITAL)) = 1 THEN 0
         WHEN SUM(DISTINCT(FLAG_FISICO)) = 1 AND SUM(DISTINCT(FLAG_DIGITAL)) = 1 THEN (SUM(FLAG_FISICO) / (SUM(FLAG_FISICO) + SUM(FLAG_DIGITAL)))
         ELSE NULL END
    """
    consulta = f"""
    WITH compra_dia AS (
        SELECT 
            CLIENTECODIGO,
            FECHACOMPRA,
            SUM(DISTINCT(FLAG_FISICO)) AS FLAG_FISICO,
            SUM(DISTINCT(FLAG_DIGITAL)) AS FLAG_DIGITAL
        FROM {tabla} 
        WHERE {restriccion}
        GROUP BY CLIENTECODIGO, FECHACOMPRA)
    SELECT 
        CLIENTECODIGO,
        {condicion_canal} AS CANAL,
        {condicion_frecuencia_canal} AS PROP_CANAL_FISICO
    FROM compra_dia
    GROUP BY CLIENTECODIGO
    """
    df = pd.read_sql(consulta, con=NZSQL_DIRECCIONRIESGOS_connection)
    df["PROP_CANAL_FISICO"] = round(df["PROP_CANAL_FISICO"], 2)
    return df

### Social demografico

In [ ]:
def get_sociodemografico(
    fecha_de_corte,
    tabla: str = TABLA_SOCIODEMOGRAFICA,
    NZSQL_DIRECCIONRIESGOS_connection=NZSQL_DIRECCIONRIESGOS_connection,
    restriccion: str = None
):
    if restriccion:
        consulta = f"""
        SELECT 
            NUMCLIENTE,
            FECHANACIMIENTO,
            GENERO,
            ESTADOCIVIL,
            ESCOLARIDADCODIGO,
            NUMDEDEPENDIENTES,
            ESTADOCODIGO,
            TIPOVIVIENDA,
            MONTOINGRESOMENSUAL,
            TAMCIUDAD,
            TIEMPOTRABAJO,
            TIEMPOVIVIENDA,
            NUMEROHIJOS,
            FLAGTELEFONOVC,
            FLAGTELEFONOCELULARVC,
            FLAGCORREOELECTRONICOVC,
            FECHAALTA 
        FROM {tabla}
        WHERE {restriccion}
        """
    else:
        consulta = f"""
        SELECT 
            NUMCLIENTE,
            FECHANACIMIENTO,
            GENERO,
            ESTADOCIVIL,
            ESCOLARIDADCODIGO,
            NUMDEDEPENDIENTES,
            ESTADOCODIGO,
            TIPOVIVIENDA,
            MONTOINGRESOMENSUAL,
            TAMCIUDAD,
            TIEMPOTRABAJO,
            TIEMPOVIVIENDA,
            NUMEROHIJOS,
            FLAGTELEFONOVC,
            FLAGTELEFONOCELULARVC,
            FLAGCORREOELECTRONICOVC,
            FECHAALTA 
        FROM {tabla}
        """
    df = pd.read_sql(consulta, con=NZSQL_DIRECCIONRIESGOS_connection)
    
    df = df.rename(columns={"NUMCLIENTE": "CLIENTECODIGO"})
    
    # Calcular antiguedades
    df["edad"] = (
        pd.to_datetime(fecha_de_corte) - pd.to_datetime(df["FECHANACIMIENTO"])
    ).dt.days / 365
    df["edad"] = df["edad"].fillna(0).astype("int")
    df["dias_de_antiguedad"] = (
        pd.to_datetime(fecha_de_corte) - pd.to_datetime(df["FECHAALTA"])
    ).dt.days
    df["dias_de_tiempotrabajo"] = (
        pd.to_datetime(fecha_de_corte) - pd.to_datetime(df["TIEMPOTRABAJO"])
    ).dt.days
    df["dias_de_tiempovivienda"] = (
        pd.to_datetime(fecha_de_corte) - pd.to_datetime(df["TIEMPOVIVIENDA"])
    ).dt.days

    df.drop(
        ["FECHAALTA", "TIEMPOTRABAJO", "TIEMPOVIVIENDA", "FECHANACIMIENTO"],
        axis=1,
        inplace=True,
    )
    return df

### Categorias

In [ ]:
def get_categorias(
    restriccion: str,
    tabla: str = TABLA_VENTAS,
    NZSQL_DIRECCIONRIESGOS_connection= NZSQL_DIRECCIONRIESGOS_connection,
):
    # Se excluyen las compras que tienen departamento 'Sin Descripcion'.
    # Se Obtienen las categorias mas frecuentes para cada cliente

    consulta = f"""WITH ListaCategorias AS
     (SELECT CLIENTECODIGO, DESDEPARTAMENTO, COUNT(*) AS CuentaCategoria, SUM(PRECIO_VTA) AS MontoCategoria
        FROM {tabla}
        WHERE {restriccion} AND DESDEPARTAMENTO != 'Sin Descripcion'
        GROUP BY CLIENTECODIGO, DESDEPARTAMENTO
        )
        SELECT s.CLIENTECODIGO, s.DESDEPARTAMENTO AS departamento_mas_frecuente, s.MontoCategoria, m.num_departamentos
            FROM ListaCategorias AS s
            JOIN (SELECT s.CLIENTECODIGO, MAX(s.CuentaCategoria) AS MaxCategoria, COUNT(DISTINCT DESDEPARTAMENTO) AS num_departamentos
                FROM ListaCategorias AS s
                WHERE DESDEPARTAMENTO != 'Sin Descripcion'
                GROUP BY s.CLIENTECODIGO
                ) AS m
        ON s.CLIENTECODIGO = m.CLIENTECODIGO AND s.CuentaCategoria = m.MaxCategoria"""
    df = pd.read_sql(consulta, con=NZSQL_DIRECCIONRIESGOS_connection)
    
    df = df.rename(
        columns={
            "DEPARTAMENTO_MAS_FRECUENTE": "departamento_mas_frecuente",
            "NUM_DEPARTAMENTOS":"num_departamentos",
            "MONTOCATEGORIA":"MontoCategoria"
        }
    )
    
    #Si hay empate elegir el de mayor monto
    max_montos = df.groupby(['CLIENTECODIGO']).MontoCategoria.transform(max)
    df = df.loc[df.MontoCategoria == max_montos]
    
    #Si hay empate nuevamente elegir de forma aleatoria
    df = df.drop_duplicates(subset = 'CLIENTECODIGO')
    
    # Eliminar la columna MontoCategoria
    df = df.drop(columns = 'MontoCategoria')
    
    return df

## Saldos, vencidos, abonos, intereses, recargos, linea credito real, puntualidad -X meses

In [ ]:
def preparar_df_por_cliente(
    df,
    fecha_de_corte,
    columna_anio="ANIO",
    columna_mes="MES",
    indice="CLIENTECODIGO",
):
    # A partir de una tabla de ventas por mes y cliente, genera un dataframe con un cliente por row, con sus datos agregados para cada mes, y la variable objetivo

    # Agrego el mes relativo a la fecha de corte
    df = agregar_mes_relativo(df, fecha_de_corte)

    # Con las variables objetivo construidas, elimino todas las filas de despues de la fecha de corte
    # Elimino el mes inmediatamente anterior a la fecha de corte por el delay de datos en produccion
    df = df[df["mes_relativo"] < -1]

    # Agrego los datos de los meses anteriores
    df = generar_datos_meses_anteriores(df, indice=indice)

    return df


def get_saldos_mes(
    restriccion: str,
    tabla: str = TABLA_DESERCION,
    NZSQL_DIRECCIONRIESGOS_connection=NZSQL_DIRECCIONRIESGOS_connection,
):
    consulta = f"""
    SELECT IDCTE,
        DATE_PART('MONTH',FECHACORTE) AS MES,
        DATE_PART('YEAR',FECHACORTE) AS ANIO,
        SUM(SDOACT) AS SALDOACTUAL, SUM(SDOVDO) AS SALDOVDO, SUM(INTNORMAL) AS INTERESNORMAL, SUM(INTMOR) AS INTERESMOR, SUM(LINEA_CRED_REAL) AS CREDITO, PUNTUALIDAD
        FROM {tabla} WHERE {restriccion}
        GROUP BY IDCTE, MES, ANIO, PUNTUALIDAD
        ORDER BY ANIO, MES, IDCTE
    """
    df = pd.read_sql(consulta, con=NZSQL_DIRECCIONRIESGOS_connection)
    df = df.rename(
        columns={
            "IDCTE": "CLIENTECODIGO",
            "SALDOACTUAL": "total_SDOACT",
            "SALDOVDO": "total_SDOVDO",
            "INTERESNORMAL": "total_INTNORMAL",
            "INTERESMOR": "total_INTMOR",
            "CREDITO": "total_LINEA_CRED_REAL",
            "first(PUNTUALIDAD)": "PUNTUALIDAD",
        }
    )
    return df

### Devoluciones

In [ ]:
def get_devoluciones(
    restriccion: str,
    tabla: str = TABLA_DEVOLUCIONES,
    NZSQL_DIRECCIONRIESGOS_connection=NZSQL_DIRECCIONRIESGOS_connection,
):
    consulta = (
        f"SELECT NUMCLIENTE FROM {TABLA_DEVOLUCIONES} WHERE {restriccion} GROUP BY NUMCLIENTE"
    )
    devoluciones = pd.read_sql(consulta, con=NZSQL_DIRECCIONRIESGOS_connection)

    devoluciones["devoluciones"] = True
    devoluciones = devoluciones.rename(columns={"NUMCLIENTE": "CLIENTECODIGO"})

    return devoluciones

### Dias ultima compra

# No implementado aun

def get_dias_ultima_compra(
    tabla: str = TABLA_HIST_CORTE
    FECHA_PRIMER_DIA: str = FECHA_PRIMER_DIA
    FECHA_CORTE: str = FECHA_CORTE
    buffer=2,
    campo_fecha: str = "FECHACORTE",
    NZSQL_DIRECCIONRIESGOS_connection=NZSQL_DIRECCIONRIESGOS_connection,
    
):
    fecha_min = (
        pd.to_datetime(FECHA_PRIMER_DIA) + relativedelta(months=-(buffer + 1))
    ).strftime("%Y-%m-%d")
    fecha_max = (
        pd.to_datetime(FECHA_PRIMER_DIA) + relativedelta(months=-(buffer))
    ).strftime("%Y-%m-%d")
    restriccion_fecha = (
        f"{campo_fecha} >= '{fecha_min}' AND {campo_fecha} < '{fecha_max}'"
    )    
    
    
    consulta = f"SELECT CLIENTECODIGO, F_ULT_COMP_T FROM {tabla} WHERE {restriccion_fecha}"
    print(consulta)
    
    df = pd.read_sql(consulta, con=NZSQL_DIRECCIONRIESGOS_connection)
    df["dias_desde_ultima_compra"] = (
        pd.to_datetime(FECHA_CORTE) - pd.to_datetime(df["F_ULT_COMP_T"])
    ).dt.days
    df = df.drop(columns=["F_ULT_COMP_T"])

    return df

# Dataset final

In [ ]:
import time

In [ ]:
def generar_dataset(
    TABLA_VENTAS: str = "MUESTRA_TRANSACCIONES_VENTAS",
    TABLA_HIST_CORTE: str = "MUESTRA_HISTORIAL_CORTE",
    TABLA_DESERCION: str = "MUESTRA_DATADESERCION",
    TABLA_SOCIODEMOGRAFICA: str = "MUESTRA_SOCIODEMOGRAFICA",
    TABLA_DEVOLUCIONES: str = "DEVOLUCIONES",
    FECHA_PRIMER_DIA: str = "2020-07-01",
    VARIABLES_OBJETIVO=True,
    buffer=2,
    meses_pre=12,
    meses_post=12,
    submuestra = None,
    fechas_festivas = {
    "navidad": "2019-12-25",
    "dia_madres": "2019-05-10",
    "regreso_clases": "2019-08-20",
    "semana_santa": "2020-04-12",
}
):
    """
    FECHA_PRIMER_DIA: Fecha del primer dia que se quiere predecir
    """
    t1 = time.time()

    #print('STARTING')
    #Fechas
    FECHA_CORTE = (
        pd.to_datetime(FECHA_PRIMER_DIA) + relativedelta(months=-1)
    ).strftime("%Y-%m-%d")
    FECHA_HIST_MIN = (
        pd.to_datetime(FECHA_PRIMER_DIA) + relativedelta(months=-(buffer + meses_pre))
    ).strftime("%Y-%m-%d")
    FECHA_HIST_MAX = (
        pd.to_datetime(FECHA_PRIMER_DIA) + relativedelta(months=-(buffer))
    ).strftime("%Y-%m-%d")
    MAX_FECHA_VAR_OBJETIVO = (
        pd.to_datetime(FECHA_PRIMER_DIA) + relativedelta(months=+(meses_post))
    ).strftime("%Y-%m-%d")

    #Restricciones
    restriccion_historica_tabla_ventas = get_restriccion(
        tabla = TABLA_VENTAS, fecha_min=FECHA_HIST_MIN, fecha_max=FECHA_HIST_MAX, submuestra = submuestra, campo_cliente = 'CLIENTECODIGO'
    )
    
    restriccion_sociodemografica = get_restriccion(tabla = TABLA_SOCIODEMOGRAFICA, submuestra = submuestra, campo_cliente = 'NUMCLIENTE')
    
    
    restriccion_historica_desercion = get_restriccion(
        tabla = TABLA_DESERCION, fecha_min=FECHA_HIST_MIN, fecha_max=FECHA_HIST_MAX, campo_fecha = 'FECHACORTE', submuestra = submuestra, campo_cliente = 'IDCTE'
    )
    
    restriccion_historica_devolucion = get_restriccion(
        tabla = TABLA_DEVOLUCIONES, fecha_min=FECHA_HIST_MIN, fecha_max=FECHA_HIST_MAX, campo_fecha = 'FECHACORTE', submuestra = submuestra, campo_cliente = 'NUMCLIENTE'
    )

    # Transacciones Ventas
    if VARIABLES_OBJETIVO:
        restriccion_tiempo_total = get_restriccion(
        fecha_min=FECHA_HIST_MIN, fecha_max=MAX_FECHA_VAR_OBJETIVO, submuestra= submuestra, campo_cliente = 'CLIENTECODIGO'
        )
        
        print("Generando monto de compras", time.time()-t1)
        df_vars_montos_compras = get_monto_compra_numero_productos_mes(
            tabla=TABLA_VENTAS, restriccion=restriccion_tiempo_total
        )
        

        print("Generando agregaciones por mes de compras",time.time()-t1)
        df_final_ventas = preparar_df_ventas(
            df=df_vars_montos_compras, variables_objetivo=VARIABLES_OBJETIVO, fecha_de_corte = FECHA_CORTE
        )


    else:
        restriccion_historica_compra = get_restriccion(
        fecha_min=FECHA_HIST_MIN, fecha_max=FECHA_HIST_MAX, submuestra = submuestra, campo_cliente = 'CLIENTECODIGO'
        )
        
        print("Generando monto de compras", time.time()-t1)
        df_vars_montos_compras = get_monto_compra_numero_productos_mes(
            tabla=TABLA_VENTAS, restriccion=restriccion_historica_compra
        )

        print("Generando agregaciones por mes de compras", time.time()-t1)
        df_final_ventas = preparar_df_ventas(
            df=df_vars_montos_compras, variables_objetivo=VARIABLES_OBJETIVO, fecha_de_corte = FECHA_CORTE
        )

    # Canal cliente
    print("Generando canales por cliente", time.time()-t1)
    df_canal_cliente_prop = get_cliente_canal_venta_prop(
        tabla=TABLA_VENTAS, restriccion=restriccion_historica_tabla_ventas
    )

    # Social demografico    
    print("Generando datos social demograficos",time.time()-t1)
    df_socioeconomico = get_sociodemografico(tabla = TABLA_SOCIODEMOGRAFICA, fecha_de_corte=FECHA_CORTE, restriccion = restriccion_sociodemografica)

    # Categorias
    print("Generando datos de categorias",time.time()-t1)
    df_categorias = get_categorias(
        tabla=TABLA_VENTAS, restriccion=restriccion_historica_tabla_ventas
    )

    # Saldos
    print("Generando datos de saldos", time.time()-t1)
    df_vars_desercion = get_saldos_mes(
        tabla=TABLA_DESERCION, restriccion=restriccion_historica_desercion
    )
    df_vars_desercion_cliente = preparar_df_por_cliente(
        df_vars_desercion, fecha_de_corte=FECHA_CORTE
    )

    # Devoluciones
    print("Generando dataset devoluciones", time.time()-t1)
    devoluciones_df = get_devoluciones(
        tabla=TABLA_DEVOLUCIONES, restriccion=restriccion_historica_devolucion
    )
    
    
    # Fechas feriados
    print("Generando dataset fechas", time.time()-t1)
    df_fechas_festivas = get_variables_fechas_relevantes(fechas = fechas_festivas, restriccion_historica_compra = restriccion_historica_tabla_ventas)
    
    
    # Dataset obtenido al hacer join en todos los df
    print("Generando dataset final",time.time()-t1)
    df_final = pd.merge(
        df_final_ventas, df_canal_cliente_prop, how="inner", on="CLIENTECODIGO"
    )
    df_final = pd.merge(df_final, df_socioeconomico, how="inner", on="CLIENTECODIGO")
    df_final = pd.merge(df_final, df_categorias, how="left", on="CLIENTECODIGO")
    df_final = pd.merge(
        df_final, df_vars_desercion_cliente, how="left", on="CLIENTECODIGO"
    )
    df_final = pd.merge(df_final, devoluciones_df, how="left", on="CLIENTECODIGO")
    df_final["devoluciones"] = df_final["devoluciones"].fillna(False)
    df_final = pd.merge(df_final, df_fechas_festivas, how = "left", on = "CLIENTECODIGO")
    
    return df_final

In [ ]:
def aplicar_filtro(
    df_final,
    TABLA_DESERCION: str = "MUESTRA_DATADESERCION",
    lista_estatus: str = "'Activos Sin Vencido', 'Vencidos 1', 'Nunca 0-15', 'Saldados 0-15'",
    FECHA_PRIMER_DIA: str = "2020-07-01",
    buffer=2,
    campo_fecha: str = "FECHACORTE",
    NZSQL_DIRECCIONRIESGOS_connection=NZSQL_DIRECCIONRIESGOS_connection,
    
):
    """ Funcion para filtrar los clientes de acuerdo a lo propuesto para el sprint 3
    """

    fecha_min = (
        pd.to_datetime(FECHA_PRIMER_DIA) + relativedelta(months=-(buffer + 1))
    ).strftime("%Y-%m-%d")
    fecha_max = (
        pd.to_datetime(FECHA_PRIMER_DIA) + relativedelta(months=-(buffer))
    ).strftime("%Y-%m-%d")
    restriccion_fecha = (
        f"{campo_fecha} >= '{fecha_min}' AND {campo_fecha} < '{fecha_max}'"
    )

    consulta = f"""SELECT IDCTE,  ESTATUSMIGRACION
            FROM {TABLA_DESERCION}
            WHERE {restriccion_fecha} 
            AND ESTATUSMIGRACION IN ({lista_estatus}) 
            GROUP BY IDCTE, ESTATUSMIGRACION"""
    df = pd.read_sql(consulta, con=NZSQL_DIRECCIONRIESGOS_connection)
    df = df.rename(columns = {'IDCTE':'CLIENTECODIGO'})
    
    df_entrenamiento_filtro = pd.merge(df_final, df, how = 'inner')
    df_entrenamiento_filtro = df_entrenamiento_filtro.drop(columns = 'ESTATUSMIGRACION')
    return df_entrenamiento_filtro

In [ ]:
months = pd.date_range("2019-07", "2020-07", freq ='MS' )
fechas_festivas = {
    "navidad": ["2018-12-25", "2019-12-25", "2020-12-25"],
    "dia_madres": ["2018-05-10","2019-05-10","2020-05-10"],
    "regreso_clases": ["2018-08-20","2019-08-26","2020-08-30"],
    "semana_santa": ["2018-04-01", "2019-04-21", "2020-04-12"],
}
df_fechas_festivas = pd.DataFrame.from_dict(fechas_festivas).astype('datetime64[ns]')

In [ ]:
def get_fechas_festivas(df_fechas_festivas, fecha_primer_dia, buffer =2, meses_pre = 12):
    condicion = (df_fechas_festivas < m) & (df_fechas_festivas > m - relativedelta(months=(buffer + meses_pre)))
    return df_fechas_festivas[condicion].melt() \
                                        .dropna() \
                                        .set_index('variable').astype(str) \
                                        .value.to_dict()
    

In [ ]:
print('STARTING')
t1 = time.time()

df = []
for m  in months:
    
    print("Processing", m, time.time()-t1)
    
    ff = get_fechas_festivas(df_fechas_festivas, m)
    print(ff)
    df_month =  generar_dataset(
        TABLA_VENTAS = "MUESTRA_TRANSACCIONES_VENTAS",
        TABLA_HIST_CORTE = "MUESTRA_HISTORIAL_CORTE",
        TABLA_DESERCION = "MUESTRA_DATADESERCION",
        TABLA_SOCIODEMOGRAFICA = "MUESTRA_SOCIODEMOGRAFICA",
        TABLA_DEVOLUCIONES = "DEVOLUCIONES",
        FECHA_PRIMER_DIA = m,
        VARIABLES_OBJETIVO=True,
        buffer=2,
        meses_pre=12,
        meses_post=12,
        submuestra = 2,
        fechas_festivas = ff
    )
    print('Filtrando churners involuntarios')
    df_month = aplicar_filtro(
        df_month,
        TABLA_DESERCION = "MUESTRA_DATADESERCION",
        lista_estatus = "'Activos Sin Vencido', 'Vencidos 1', 'Nunca 0-15', 'Saldados 0-15'",
        FECHA_PRIMER_DIA = m,
        buffer=2,
        campo_fecha = "FECHACORTE",
        NZSQL_DIRECCIONRIESGOS_connection=NZSQL_DIRECCIONRIESGOS_connection,
    )
    df.append(df_month)
    
print('concatenando meses...')
df = pd.concat(df)   

STARTING
Processing 2019-07-01 00:00:00 0.0004792213439941406
{'navidad': '2018-12-25', 'dia_madres': '2019-05-10', 'regreso_clases': '2018-08-20', 'semana_santa': '2019-04-21'}
Generando monto de compras 0.0003829002380371094
Generando agregaciones por mes de compras 296.2789969444275
Generando canales por cliente 298.282812833786
Generando datos social demograficos 307.87141013145447
Generando datos de categorias 520.8761978149414
Generando datos de saldos 533.2948966026306
Generando dataset devoluciones 810.1661105155945
Generando dataset fechas 810.8061430454254
Generando dataset final 849.9321048259735
DONE! Duracion: 851.1893074512482
Filtrando churners involuntarios
Processing 2019-08-01 00:00:00 934.3486957550049
{'navidad': '2018-12-25', 'dia_madres': '2019-05-10', 'regreso_clases': '2018-08-20', 'semana_santa': '2019-04-21'}
Generando monto de compras 0.0002739429473876953
Generando agregaciones por mes de compras 313.12663531303406
Generando canales por cliente 315.426425218

KeyboardInterrupt: 

In [ ]:
from project_lib import Project
project = Project.access()
print('guardando dataset...')
project.save_data(file_name = "df_entrenamiento_filtro_25_rolling_window.csv",data = df.to_csv(index=False))
print("DONE!", 'Duracion:',time.time()-t1)